In [230]:
import pandas as pd
import numpy as np
import requests
import csv

In [232]:
ALP_df   = pd.read_csv("ALP_2017_Wei.csv")
MSSMD_df = pd.read_csv("MSSMD_2017_Wei.csv")
MSSMD_df_newGP = pd.read_csv("MSSMD_patchedGridpacks.csv")
NMSSM_df = pd.read_csv("NMSSM_2017_Wei.csv")

# Same as MSSMD, but different dataset name. Used for updating correctly old requests 
DarkSUSY_df    = pd.read_csv('Dark_SUSY_2017.csv')

In [233]:
MSSMD_gp      = !grep -o "MSSMD_\w*CMSSW_7_1_30_tarball.tar.xz$" MSSMD_MG242_cvmfs_paths.txt
MSSMD_gp_path = !grep -o "/cvmfs/cms.cern.ch/phys_generator/gridpacks/2017/13TeV/madgraph/V5_2.\d.\d/MSSMD_\w*/v[1-9]/MSSMD_\w*CMSSW_7_1_30_tarball.tar.xz$" MSSMD_MG242_cvmfs_paths.txt

ALP_gp      = !grep -o "ALP_\w*CMSSW_7_1_30_tarball.tar.xz$" ALP_MG242_cvmfs_paths.txt
ALP_gp_path = !grep -o "/cvmfs/cms.cern.ch/phys_generator/gridpacks/2017/13TeV/madgraph/V5_2.\d.\d/ALP_\w*/v[1-9]/ALP_\w*CMSSW_7_1_30_tarball.tar.xz$" ALP_MG242_cvmfs_paths.txt

## MSSMD requests

In [234]:
MSSMD_dict = { 'gridpack': MSSMD_gp,
               'gridpack_path' : MSSMD_gp_path}
MSSMD_gp_df = pd.DataFrame(MSSMD_dict)

In [235]:
new_columns = MSSMD_df.columns.values
new_columns[3] = 'gridpack'
MSSMD_df.columns = new_columns

In [236]:
# Check if the indexing btw the to DF is the same
match = 0
for ix_l, l in enumerate(MSSMD_df.gridpack):
    for ix_r, r in enumerate(MSSMD_gp_df.gridpack):
        if l == r:
            match=match+1
            print("-"*60)
            print('%d - %d' % (ix_l, ix_r))
            print("-"*60)
# It is NOT

------------------------------------------------------------
0 - 4
------------------------------------------------------------
------------------------------------------------------------
1 - 0
------------------------------------------------------------
------------------------------------------------------------
2 - 1
------------------------------------------------------------
------------------------------------------------------------
3 - 2
------------------------------------------------------------
------------------------------------------------------------
4 - 3
------------------------------------------------------------
------------------------------------------------------------
5 - 7
------------------------------------------------------------
------------------------------------------------------------
6 - 9
------------------------------------------------------------
------------------------------------------------------------
7 - 10
------------------------------------

In [237]:
MSSMD_df_final = pd.merge(MSSMD_df, MSSMD_gp_df, on='gridpack')

In [238]:
import re

class McM_Request:
    def __init__(self, pdSeries, fragment_name=''):
        self.dataset_name  = pdSeries['MC Sample name'].replace('13TeV','TuneCP5_13TeV-madgraph-pythia8')
        self.total_events  = int(pdSeries['Events'].replace(',',''))
        self.gridpack_path = pdSeries['gridpack_path'].replace('\n','')
        self.generators    = pdSeries['Generator'].replace(' + ','+').replace(' ','_').replace('+',' ')
        self.notes = self._note()
        if fragment_name:
            self.fragment = self._modFragment(fragment_name)
        else:
            self.fragment = ''
        
    def _note(self):
        return self.dataset_name.replace('_',' ')
        #re_par1 = re.compile('Mneu1_(.*?)(?=_)')
        #re_par2 = re.compile('MAD_(.*?)(?=_)')
        #re_par3 = re.compile('cT_(.*?)(?=_)')

        #par1 = re_par1.search(self.dataset_name).group().split('_')[1]
        #par2 = re_par2.search(self.dataset_name).group().split('_')[1].replace('p','.')
        #par3 = re_par3.search(self.dataset_name).group().split('_')[1].replace('p','.')

        #note = "MSSMD Mneu1 = {0} MAD = {1} cTau = {2}".format(par1, par2, par3) 
        #return note
    
    def _modFragment(self, fragment_name):
        return fragment_name.replace('__GRIDPACK__', self.gridpack_path)
            
    def __str__(self):
        string = "="*30+"\n"
        string += ("Dataset name: %s" %self.dataset_name)+"\n"
        string += ("Events: %d" %self.total_events)+"\n"
        string += ("gridpack path: %s" %self.gridpack_path)+"\n"
        string += "\n\n"
        string += ("notes: %s" %self.notes)+"\n"
        string += ("fragment: %s" %self.fragment)+"\n"
        string += ("generators: %s" %self.generators)+"\n"
        string += "="*30+"\n"
        return string

In [141]:
MSSMD_pythia_frag_url = MSSMD_df_final.loc[0][4]
MSSMD_pythia_frag_raw_url = MSSMD_pythia_frag_url.replace('github.com','raw.githubusercontent.com').replace('/blob','')

# Changing pythia Tune
r = requests.get(MSSMD_pythia_frag_raw_url)
fragment_oldTune = r.text
oldTune_conf = 'Configuration.Generator.Pythia8CUEP8M1Settings_cfi'
newTune_conf = 'Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi'
oldTune_proc = 'pythia8CUEP8M1SettingsBlock'
newTune_proc = 'pythia8CP5SettingsBlock'
oldTune_seq  = 'pythia8CUEP8M1Settings'
newTune_seq  = 'pythia8CP5Settings'
MSSMD_fragment = fragment_oldTune.replace(oldTune_conf, newTune_conf).replace(oldTune_proc, newTune_proc).replace(oldTune_seq, newTune_seq)
print(MSSMD_fragment)

## Pythia8 fragment for 13TeV collisions
import FWCore.ParameterSet.Config as cms

from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *

generator = cms.EDFilter("Pythia8HadronizerFilter",
    pythiaHepMCVerbosity = cms.untracked.bool(False),
    maxEventsToPrint = cms.untracked.int32(0),
    pythiaPylistVerbosity = cms.untracked.int32(1),
    filterEfficiency = cms.untracked.double(1.0),
    crossSection = cms.untracked.double(0.0),
    comEnergy = cms.double(13000.0),
    PythiaParameters = cms.PSet(
        pythia8CommonSettingsBlock,
        pythia8CP5SettingsBlock,
        parameterSets = cms.vstring(
            'pythia8CommonSettings',
            'pythia8CP5Settings',
        )
    )
)



In [142]:
# url_cards = 'https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/MSSMD'

# for index, row in MSSMD_df_final.iterrows():
#     LHEproducer = """import FWCore.ParameterSet.Config as cms

# externalLHEProducer = cms.EDProducer("ExternalLHEProducer",
#     args = cms.vstring('__GRIDPACK__'),
#     nEvents = cms.untracked.uint32(5000),
#     numberOfParameters = cms.uint32(1),
#     outputFile = cms.string('cmsgrid_final.lhe'),
#     scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
# )

# #Link to datacards:
# #{0}
# """.format(url_cards)
#     
#     index = MSSMD_fragment.find('from Configuration.Generator.Pythia8CommonSettings_cfi')
#     MSSMD_fragment_full = LHEproducer + MSSMD_fragment[index]
#     req1 = McM_Request(row, MSSMD_fragment_full)
#     print(req1)

Dataset name: MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0_TuneCP5_13TeV-madgraph-pythia8
Events: 100000
gridpack path: /cvmfs/cms.cern.ch/phys_generator/gridpacks/2017/13TeV/madgraph/V5_2.4.2/MSSMD_Mneu1_10_MAD_0p25_cT_0/v1/MSSMD_Mneu1_10_MAD_0p25_cT_0_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz


notes: MSSMD mH 125 mN1 10 mGammaD 0p25 cT 0 TuneCP5 13TeV-madgraph-pythia8
fragment: import FWCore.ParameterSet.Config as cms

externalLHEProducer = cms.EDProducer("ExternalLHEProducer",
    args = cms.vstring('/cvmfs/cms.cern.ch/phys_generator/gridpacks/2017/13TeV/madgraph/V5_2.4.2/MSSMD_Mneu1_10_MAD_0p25_cT_0/v1/MSSMD_Mneu1_10_MAD_0p25_cT_0_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)

#Link to datacards:
#https://github.com/cms-sw/genproductions/tree/master/

In [239]:
DarkSUSY_df    = pd.read_csv('Dark_SUSY_2017.csv')
DarkSUSY_df.columns = new_columns
DarkSUSY_df_merged = pd.merge(MSSMD_df_final, DarkSUSY_df, on='gridpack')

In [240]:
DarkSUSY_df_merged['Dataset name'] = DarkSUSY_df_merged['MC Sample name_y'].apply(lambda x: x.replace('13TeV','TuneCP5_13TeV-madgraph-pythia8'))
DarkSUSY_df_merged

DarkSUSY_PrepID_df = pd.read_csv('DarkPhotons_PrepID_list.csv')
Final_MSSMD_df = pd.merge(DarkSUSY_df_merged,DarkSUSY_PrepID_df, on='Dataset name')

In [241]:
Final_MSSMD_df.drop(['MC Sample name_y','Events_y', 'Generator_y',
       'Pythia8 fragment (Same for all samples)_y', 'Dataset name',], axis=1, inplace=True)
#Final_MSSMD_df.loc[Final_MSSMD_df['MC Sample name_x']=='MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0_13TeV','PrepID']
Final_MSSMD_df.columns = [x.replace('_x','') for x in Final_MSSMD_df.columns ]
Final_MSSMD_df

,MC Sample name,Events,Generator,gridpack,Pythia8 fragment (Same for all samples),gridpack_path,PrepID
0,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02482
1,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0p05_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0p05_slc6_amd64_gcc...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02483
2,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0p1_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0p1_slc6_amd64_gcc4...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02484
3,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0p2_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0p2_slc6_amd64_gcc4...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02485
4,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0p5_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0p5_slc6_amd64_gcc4...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02486
5,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_1_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_1_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02487
6,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_2_13TeV,"300,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_2_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02488
7,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_3_13TeV,"300,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_3_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02489
8,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_5_13TeV,"1,000,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_5_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02490
9,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_10_13TeV,"5,000,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_10_slc6_amd64_gcc48...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02491


In [242]:
Final_MSSMD_Patched_df = pd.merge(Final_MSSMD_df,MSSMD_df_newGP, on='gridpack')

,MC Sample name,Events,Generator,gridpack,Pythia8 fragment (Same for all samples),gridpack_path_x,PrepID,gridpack_path_y
0,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02482,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
1,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0p05_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0p05_slc6_amd64_gcc...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02483,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
2,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0p1_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0p1_slc6_amd64_gcc4...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02484,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
3,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0p2_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0p2_slc6_amd64_gcc4...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02485,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
4,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0p5_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0p5_slc6_amd64_gcc4...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02486,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
5,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_1_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_1_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02487,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
6,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_2_13TeV,"300,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_2_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02488,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
7,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_3_13TeV,"300,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_3_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02489,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
8,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_5_13TeV,"1,000,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_5_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02490,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
9,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_10_13TeV,"5,000,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_10_slc6_amd64_gcc48...,https://github.com/weishi10141993/DarkSector/b...,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...,HIG-RunIIFall17wmLHEGS-02491,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...


In [246]:
Final_MSSMD_Patched_df.drop(['gridpack_path_x'], axis=1, inplace=True)
#Final_MSSMD_df.loc[Final_MSSMD_df['MC Sample name_x']=='MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0_13TeV','PrepID']
Final_MSSMD_Patched_df.columns = [x.replace('_y','') for x in Final_MSSMD_Patched_df.columns ]
Final_MSSMD_Patched_df

,MC Sample name,Events,Generator,gridpack,Pythia8 fragment (Same for all samples),PrepID,gridpack_path
0,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,HIG-RunIIFall17wmLHEGS-02482,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
1,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0p05_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0p05_slc6_amd64_gcc...,https://github.com/weishi10141993/DarkSector/b...,HIG-RunIIFall17wmLHEGS-02483,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
2,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0p1_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0p1_slc6_amd64_gcc4...,https://github.com/weishi10141993/DarkSector/b...,HIG-RunIIFall17wmLHEGS-02484,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
3,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0p2_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0p2_slc6_amd64_gcc4...,https://github.com/weishi10141993/DarkSector/b...,HIG-RunIIFall17wmLHEGS-02485,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
4,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_0p5_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_0p5_slc6_amd64_gcc4...,https://github.com/weishi10141993/DarkSector/b...,HIG-RunIIFall17wmLHEGS-02486,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
5,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_1_13TeV,"100,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_1_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,HIG-RunIIFall17wmLHEGS-02487,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
6,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_2_13TeV,"300,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_2_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,HIG-RunIIFall17wmLHEGS-02488,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
7,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_3_13TeV,"300,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_3_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,HIG-RunIIFall17wmLHEGS-02489,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
8,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_5_13TeV,"1,000,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_5_slc6_amd64_gcc481...,https://github.com/weishi10141993/DarkSector/b...,HIG-RunIIFall17wmLHEGS-02490,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...
9,MSSMD_mH_125_mN1_10_mGammaD_0p25_cT_10_13TeV,"5,000,000",MG5_aMC v2.4.2 + Pythia8,MSSMD_Mneu1_10_MAD_0p25_cT_10_slc6_amd64_gcc48...,https://github.com/weishi10141993/DarkSector/b...,HIG-RunIIFall17wmLHEGS-02491,/cvmfs/cms.cern.ch/phys_generator/gridpacks/20...


In [248]:
url_cards = 'https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/MSSMD'

with open('Wei_MSSMD_patched.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['Dataset name','PrepID','fragment','notes','Generator'])
    
    for index, row in Final_MSSMD_Patched_df.iterrows():
        LHEproducer = """import FWCore.ParameterSet.Config as cms
                            
externalLHEProducer = cms.EDProducer("ExternalLHEProducer",
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
    )

#Link to datacards:
#{0}
""".format(url_cards)
        
        index = MSSMD_fragment.find('from Configuration.Generator.Pythia8CommonSettings_cfi')
        MSSMD_fragment_full = LHEproducer + "\n\n" +MSSMD_fragment[index:]
        req1 = McM_Request(row, MSSMD_fragment_full)
        csvwriter.writerow([req1.dataset_name, row[-2], req1.fragment, req1.notes, req1.generators])

In [229]:
print(MSSMD_fragment_full)


import FWCore.ParameterSet.Config as cms
                            
externalLHEProducer = cms.EDProducer("ExternalLHEProducer",
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
    )

#Link to datacards:
#https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/MSSMD


from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *

generator = cms.EDFilter("Pythia8HadronizerFilter",
    pythiaHepMCVerbosity = cms.untracked.bool(False),
    maxEventsToPrint = cms.untracked.int32(0),
    pythiaPylistVerbosity = cms.untracked.int32(1),
    filterEfficiency = cms.untracked.double(1.0),
    crossSection = cms.untracked.double(0.0),
    comEnergy = cms

## ALP requests

In [123]:
LHEproducer = """import FWCore.ParameterSet.Config as cms

externalLHEProducer = cms.EDProducer("ExternalLHEProducer",
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)

#Link to datacards:
#https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/ALP
"""


ALP_dict = { 'gridpack': ALP_gp,
             'gridpack_path' : ALP_gp_path}
ALP_gp_df = pd.DataFrame(ALP_dict)

pythia_frag_url = ALP_df.loc[0][4]
pythia_frag_raw_url = pythia_frag_url.replace('github.com','raw.githubusercontent.com').replace('/blob','')

r = requests.get(pythia_frag_raw_url)
fragment = r.text

index = fragment.find('from Configuration.Generator.Pythia8CommonSettings_cfi')
fragment_full = LHEproducer+'\n\n'+fragment[index:]
print(fragment_full)

import FWCore.ParameterSet.Config as cms

externalLHEProducer = cms.EDProducer("ExternalLHEProducer",
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)

#Link to datacards:
#https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/ALP


from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *

generator = cms.EDFilter("Pythia8HadronizerFilter",
    pythiaHepMCVerbosity = cms.untracked.bool(False),
    maxEventsToPrint = cms.untracked.int32(0),
    pythiaPylistVerbosity = cms.untracked.int32(1),
    filterEfficiency = cms.untracked.double(1.0),
    crossSection = cms.untracked.double(0.0),
    comEnergy = cms.double(13000.0),
    PythiaParame

In [71]:
new_columns = ALP_df.columns.values
new_columns[3] = 'gridpack'
ALP_df.columns = new_columns

In [76]:
ALP_df_final = pd.merge(ALP_df, ALP_gp_df, on='gridpack')

In [116]:
with open('Wei_ALP.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['Dataset name','Events','fragment','notes','Generator'])
    
    for index, row in ALP_df_final.iterrows():
        req1 = McM_Request(row, fragment_full)
        csvwriter.writerow([req1.dataset_name,req1.total_events,req1.fragment, req1.notes, req1.generators])

## NMSSM requests

In [124]:
pythia_frag_url = NMSSM_df.loc[0][3]
pythia_frag_raw_url = pythia_frag_url.replace('github.com','raw.githubusercontent.com').replace('/blob','')

# Changing pythia Tune
r = requests.get(pythia_frag_raw_url)
fragment_oldTune = r.text
oldTune_conf = 'Configuration.Generator.Pythia8CUEP8M1Settings_cfi'
newTune_conf = 'Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi'
oldTune_proc = 'pythia8CUEP8M1SettingsBlock'
newTune_proc = 'pythia8CP5SettingsBlock'
oldTune_seq  = 'pythia8CUEP8M1Settings'
newTune_seq  = 'pythia8CP5Settings'
fragment = fragment_oldTune.replace(oldTune_conf, newTune_conf).replace(oldTune_proc, newTune_proc).replace(oldTune_seq, newTune_seq)
print(fragment)

# NMSSM Hto2Ato4mu 13TeV pythia8 configuration file
import FWCore.ParameterSet.Config as cms

from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *

generator = cms.EDFilter("Pythia8GeneratorFilter",
                         pythiaHepMCVerbosity = cms.untracked.bool(False),
                         maxEventsToPrint = cms.untracked.int32(0),
                         pythiaPylistVerbosity = cms.untracked.int32(1),
                         filterEfficiency = cms.untracked.double(1.0),
                         crossSection = cms.untracked.double(0.0),
                         comEnergy = cms.double(13000.0),
                         PythiaParameters = cms.PSet(
        pythia8CommonSettingsBlock,
        pythia8CP5SettingsBlock,
        pythiaUESettings = cms.vstring(),
        processParameters = cms.vstring(
            # This section should be entirely in Pythia 8. See details in
                    #   - 

In [120]:
NMSSM_df.loc[0]

Dataset name                        NMSSM_HToAATo4Mu_mH_90_mA_0p25_13TeV
Total events                                                     200,000
Generator                                                        Pythia8
Generator fragment     https://github.com/weishi10141993/DarkSector/b...
Name: 0, dtype: object

In [136]:
with open('Wei_NMSSM.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['Dataset name','Events','fragment','notes','Generator'])
    for index, row in NMSSM_df.iterrows():
        datasetname = row[0].replace('13TeV','TuneCP5_13TeV-pythia8')
        events = int(row[1].replace(',',''))
        generator = row[2]
        notes = datasetname.replace('_',' ')
        pythia_frag_url = row[3]
        pythia_frag_raw_url = pythia_frag_url.replace('github.com','raw.githubusercontent.com').replace('/blob','')
        r = requests.get(pythia_frag_raw_url)
        fragment_oldTune = r.text + '\n\nProductionFilterSequence = cms.Sequence(generator)'
        fragment = fragment_oldTune.replace(oldTune_conf, newTune_conf).replace(oldTune_proc, newTune_proc).replace(oldTune_seq, newTune_seq)
        
        csvwriter.writerow([datasetname,events,fragment,notes,generator])
        #print('%s - %d - \nFragment: %s \nNote: %s - gen: %s'% (datasetname,events,fragment,notes,generator))